In [ ]:
#William Craig
#MFG 598
#HW3
#February 8, 2023

In [57]:
# Qn 1 (10 points)
# Read the file ‘grades.csv’. Write code that finds and displays the 5 student IDs 
# and their corresponding grades who received the lowest grades in the class. 
# Some students have not appeared for the exam, but they are not to be treated as 0. 
# They need to be simply skipped from the computation. (20points)


import zipfile
import csv

#This code extracts the contents of a zipped file into a working directory.

#An alternative is to work within the zipped file, but this code is also easier to understand.

with zipfile.ZipFile("C:\\Code\\MFG598\\Homework\\Resources\\HW3_Data_Files.zip", "r") as unzipped_file:
    unzipped_file.extractall()

#This simply opens a csv from the working directory.

with open("grades.csv", "r") as csv_file:
    #Extracts contents of the file in the format of a dictionary. 
    csv_contents = csv.DictReader(csv_file)
    nonzero_dict = {}

#This code iterates through the contents of a the new dictionary and appends them to a dictionary if the score is nonzero. 
#   This is accomplished by 'if row["Scores"]:", where it is implied that the 'True' result entails a value existing. 
    for row in csv_contents:
        if row["Scores"]:
            #This opportunity is taken to turn scores into integers, so that they can be sorted later. Strings can be sorted as well, but this is more intuitive.
            nonzero_dict[row['Student IDs']] = int(row['Scores'])

#Sorting is difficult with libraries, since they are by definition unordered.
#Here, we essentially convert the scores column into a list, associate each value with a key using lambda, then place the contents back into a new dictionary in sequence.
    sorted_nonzero_dict = dict(sorted(nonzero_dict.items(), key=lambda item: item[1]))

#Again, we cannot print the highest or lowest values of a dictionary. Therefore, we print the first five values of a sorted list.
    sorted_nonzero_dict_lowest_five = list(sorted_nonzero_dict.items())[:5]

    print(f"The lowest 5 non-zero scores and their associated values are: \n {sorted_nonzero_dict_lowest_five}")

The lowest 5 non-zero scores and their associated values are: 
 [('103', 56), ('109', 66), ('106', 75), ('104', 78), ('113', 86)]


In [ ]:
# Qn 2 (20 points)
# Open and read the contents of the file – ‘2_NoofParts_assem.txt’. Perform the following (20 points):
# a)	Calculate how many entries are available in that file excluding the header.
# b)	Calculate the sum of all parts from each file. Essentially, finding the sum of all values contained in the 2nd column of the file.
# c)	Extract the part ID that has the largest associated no. of parts from the entire list_input.


file = "2_NoofParts_assem.txt"

import zipfile
import chardet

with zipfile.ZipFile("C:\Code\MFG598\Homework\Resources\HW3_Data_Files.zip", 'r') as unzipped:
    unzipped.extractall()

#When I try to open here, I get a decode error for utf-8. What type of encoding is this file?

#"rb" does "read binary" so we can look deeply in the file.
with open("2_NoofParts_assem.txt", "rb") as file:
    result = chardet.detect(file.read())
    print(result['encoding'])

#Not sure how to integrate this into the encoding = statement below. I'm sure there's a way.

with open("2_NoofParts_assem.txt", 'r', encoding = 'utf-16') as file:
    #print(file.read())
    lines = file.readlines()
    num_lines = len(lines)

print(f'There are {num_lines - 1} lines in the file, excluding the header')

#Doing my EDA here
#print(lines) 
#Based on this print statement, our values are separated by \t and followed by \t\t\n
#There's a value to 'clean' the statement but I don't remember right now.

#Let's first split the items.

split_values = []
for item in lines:
    split_values.append(item.split("\t"))

#print(split_values)

#From this print statement, we see that the header is still included. We can take it out now or later, because it will interfere with int operations.
#We can nest the entire value-stripping code block in a 'try' statement and then exit if we have a ValueError.
#We also need to account for the fact that the list_input now consists of lists.

values_only = []
for sublist in split_values:
    for item in sublist:
        try:
            values_only.append(int(item))
        except ValueError:
            continue
#print(values_only)

print(f"The sum of all integer values in the .txt file is {sum(values_only)}")

# c)	Extract the part ID that has the largest associated no. of parts from the entire list_input.

#We need to know which entry in the list_input of lists has the max value, then we can just navigate to it.
#My instinct is to iterate through the list_input increasing an index number, but I feel like there's a more elegant way.
#I learned about custom keys and will apply that here.

#This definition gives us the key value of whatever term in the master list_input that we want.
def extract_max_num(item):
    return (item[1])

#This item finds the sublist. The value in the brackets lets us 'skip' the header, starting at 1 and going onward.
max_part = max(split_values[1:], key=extract_max_num)

#The definition acts like a lambda function in that the value of max() is used as a parameter.

part_id = max_part[0]

print(f"The part ID with the highest # of associated parts is {part_id}")
print(f"The number of associated parts for that ID is {max_part[1]}")

In [ ]:
# Qn 3 (25 points)
# Using a python script, open the files contained in the .zip file – “3_Jobs_Completed_log.zip”. 
# Scan the files for the line that starts with the word string – “Jobs Completed.. ”. Extract the number associated with this line and for all instances that this word string appears across all log files, count the total sum across all files contained within the .zip file. (20points) 

# For example:

#  	Jobs Completed.. 10 2018-09-04 08:21:28.503153

# Extract the number 10 from this sentence which signifies the total number of jobs completed at the point in time.  
# Find for all instances in which the string - ‘Jobs Completed’ appears, 
# find the total number of jobs completed by adding the numbers from across the provided log files.

import zipfile
import chardet

filetypes = []
total_sum = 0
#Opening .zip within .zip here and extracting:
with zipfile.ZipFile("C:\Code\MFG598\Homework\Resources\HW3_Data_Files.zip", 'r') as wrapper:
    with wrapper.open("Jobs_Completed_log.zip", 'r') as Jobs_Completed_file:
            with zipfile.ZipFile(Jobs_Completed_file) as Jobs_Completed_log:

#Next I'll iterate through a list_input of the files and detect encoding for each with chardet, since I have no idea if they are homoegenous.
#Is this overkill?
#To make the code resilient, I'll pass the encoding data to another 

                #I was doing this by habit, but it looks like it's not necessary if I work within the zipfile condition. 
                #   Not sure if this is preferable.
                # Jobs_Completed_log.extractall()

                Jobs_Completed_log_file_list = Jobs_Completed_log.namelist()
                for file in Jobs_Completed_log_file_list[:len(Jobs_Completed_log_file_list)]:
                    with Jobs_Completed_log.open(file) as f:
                        file_data = f.read()
                        encoding_result = chardet.detect(file_data)
                        encoding = encoding_result["encoding"]
                        filetypes.append(encoding)

                        #Found that a typical str.split does not work here like splitting a utf-8 file does.
                            # Resulted in a TypeError 
                            # Instead, I'll 'decode'.
                        file_data_str = file_data.decode(encoding)
                        lines = file_data_str.split("\n")

                        #Now that we have useful data, I'll search for the Jobs Completed lines, 
                        #   split then up, and grab the total number with an index.
                        for line in lines:
                            if line.startswith("Jobs Completed.. "):
                                parts = line.split(" ")
                                total_sum += int(parts[2])

#print("Filetypes:", filetypes)
print("Jobs completed:", total_sum)
#print(file_data)



In [54]:
# Qn 4 (20 points)
# For the same files above, calculate the following:
# a)	How many jobs were completed in the period between 15th Aug 2018 to 15th Sept 2018?
# b)	The day of the year in which the maximum number of jobs was completed.
# c)	Calculate the total number of days elapsed between the time at which the first job batch was completed to the last batch completed.

import zipfile
import chardet
import io
import re
from datetime import datetime, timedelta
import statistics

filetypes = []
total_sum = 0
master_file = []
date_list = []

date_format = '\b\n{4}-'

# Opening .zip within .zip here and extracting:
with zipfile.ZipFile("C:\Code\MFG598\Homework\Resources\HW3_Data_Files.zip", 'r') as wrapper:
    inner_zip = wrapper.open("Jobs_Completed_log.zip")
    with zipfile.ZipFile(inner_zip) as Jobs_Completed_file:
        #The code iterates through each file by creating a list of names and then selecting them one-by-one.
        for filename in Jobs_Completed_file.namelist():
            #For each file, it is opened as 'file' then an operation is performed.
            with Jobs_Completed_file.open(filename) as file:
                #TextIOWrapper is used to ensure the program reads a string instead of a stream of bytes, which results in a TypeError.
                #Readlines passes each line into a list object.
                buffer = io.TextIOWrapper(file, encoding="utf-8").readlines()
                #master_file is appended with all elements from all files. This produces a list that can be quickly searched/
                master_file.append(buffer)

#This uses regex to recognize digits 'd' of length 4, then 2, then 2. they can be separated by hyphens.
date_regex = re.compile(r'\d{4}-\d{2}-\d{2}')

#Now, for each of the 5 sublists of master_file, each list item created by .readlines() is read.
for buffer in master_file:
    for string in buffer:
        #match is a buffer item we'll append to a master date_list.
        match = date_regex.search(string)
        if match:
            date_list.append(match.group())



#Now we'll iterate through the list of dates using if conditions to establish a range.

#Creating boundaries here.
start_date = '2018-08-15'
end_date = '2018-09-15'

dates_in_range = []

for date in date_list:
    if date >= start_date:
        if date <= end_date:
            dates_in_range.append(date)
print(f"The number of entries between {start_date} and {end_date} is {len(dates_in_range)}")

#To find the year in which the most jobs were created, I'll strip the year from all date_list entries and append it to a years_only list.
#Then, we can just run statistics.mode
# years_only = []
# year = re.compile(r'\d{4}')
# for date in date_list:
#     match = year.search(string)
#     #The use of .group means that the list will only be appended with the string and not other information.
#     years_only.append(match.group())

print(f'The day with the most associated jobs is: {statistics.mode(date_list)}')

#For the total time elapsed between first and last jobs, we'll use max and min on the date_list

#We'll have to convert to a datetime object here so we can perform operations on the dates.

first_job = datetime.strptime(min(date_list), '%Y-%m-%d')
last_job = datetime.strptime(max(date_list), '%Y-%m-%d')
time_delta = last_job - first_job
#We could get rid of the time element with strftime, but it seems overkill here. 'string-from-time.'
print(f"The days elapsed between the first job and last job are {time_delta}")


The number of entries between 2018-08-15 and 2018-09-15 is 2004
The day with the most associated jobs is: 2018-08-11
The days elapsed between the first job and last job are 36 days, 0:00:00


In [55]:
# Qn 5 (25 points)
# Read the following (6_Part1.stl) triangular mesh file available in ASCII format. 
# For those of you aware of 3D printing, you will recognize this is as a .STL file. 
# For those who aren’t aware of what a .STL file is, please read up online on the format of a .STL ascii file. 
# An ASCII formatted .STL file can be opened in any text editor. 
# Every vertex of the triangle is preceeded by the word ‘vertex’ followed by the x, y and z coordinates. 
# Therefore, ONE Triangle should have 3 vertex entries. Typically, the units is not specified, 
# but you can assume it to be in millimeters. 

# Write functions that calculate the following: (20 points)
# a.	Find the total number of triangles listed in the file.
# b.	Store the coordinates of each triangle in a 3-tuple list_input containing N indices, where N is the total number of triangles.
# c.	list_input the area of each triangle in the file. Compute the total surface area of all triangles listed in the file. Read up online on the formula to calculate the area of a triangle, given three vertices.
# '''

file = "C:/Code/MFG598/Homework/6_Part1.stl"
vertex_count = 0

#Using the counting function is going to be more lightweight than using file.readlines()
    #...then iterating through the list of lines. We'll readlines later however.
with open(file, "r") as stl_file:
    file_data = stl_file.read()
    vertex_count = file_data.count("vertex")

triangle_count = vertex_count/3

print(f"The number of triangles is: {int(triangle_count)}")

#Now we're using readlines and we'll use 'vertex' as a pointer to access just the tuple of values after it.
with open(file, "r") as file:
    file_data = file.readlines()

triangles = []
for line in file_data:
    if 'vertex' in line:
        coordinates = tuple([float(x) for x in line.strip().split()[1:]])
        triangles.append(coordinates)

#Now find the total area of all triangles, and also the area of each triangle.
#I'm not sure if the question wants me to append this to list_input

area_list = []


tri = [triangles[i:i+3] for i in range(0, len(triangles), 3)]

#Going to assume that the triangle format is Ax, Ay, Az, Bx, By, Bz...
#Iterate through each grouping of vertices and then do the math for volume.
def triangle_volume(tri):
    x1, y1, z1 = tri[0]
    x2, y2, z2 = tri[1]
    x3, y3, z3 = tri[2]
    return abs(x1*y2*z3 + x2*y3*z1 + x3*y1*z2 - x1*y3*z2 - x2*y1*z3 - x3*y2*z1) / 6

volumes = [triangle_volume(triangle) for triangle in tri]
print(f"Total area of all triangles is: {sum(volumes)}. \n The areas of each triangle in the mesh follow:")
print(volumes)


The number of triangles is: 44
Total area of all triangles is: 1022629.4450895642. 
 The areas of each triangle in the mesh follow:
[0.0, 0.0, 0.0, 2638.9353534853403, 5472.944843684953, 13085.550675119106, 9744.057557894803, 0.0, 106497.19095809023, 15631.535308557655, 145880.1784239923, 145880.1784239923, 106497.19095809023, 9459.949236665683, 9459.949236665685, 28294.53112314489, 3692.9550878294663, 28294.5311231449, 26323.460256115435, 3692.9550878294663, 45158.04214259463, 45158.04214259464, 26323.460256115435, 5472.944843684953, 5472.944843684953, 2638.9353534853403, 0.0, 9933.88744898081, 2638.9353534853412, 0.0, 6537.786042777389, 9933.887448980808, 7099.877958781198, 7099.877958781198, 3566.0235234441257, 13167.256416249083, 33228.97134803277, 11531.591827811746, 28377.47583342216, 27533.410225028416, 15783.804821429241, 11188.593735626715, 14246.033970931187, 29991.567939339788]
